In [5]:
import pandas as pd

# 1. Load Excel file
file_path = "sample_claims_data.xlsx"   # <-- replace with your file
df = pd.read_excel(file_path)

print("Raw Data:")
display(df.head())

Raw Data:


,ClaimNumber,FocusCode,Pre_Reimb_Amount,Over_Payment,Decision_Type,Denial_Reason
0,1233,111,500.00,100.00,Denied,56900.0
1,1233,112,500.00,100.00,Denied,56900.0
2,2001,97110,1506.30,153.69,Denied,NaN
3,2002,97112,220.16,0.00,Denied,NaN
4,2003,97116,17.28,0.00,Denied,NaN


In [6]:
# 2. Apply Business Rules
df_filtered = df.copy()

# Exclude Exclusion / Other
df_filtered = df_filtered[~df_filtered['Decision_Type'].isin(['Exclusion', 'Other'])]

# 56900 denial reason is included automatically (no exclusion applied)
print("After Business Rule Filters:")
display(df_filtered)

After Business Rule Filters:


,ClaimNumber,FocusCode,Pre_Reimb_Amount,Over_Payment,Decision_Type,Denial_Reason
0,1233,111,500.00,100.00,Denied,56900.0
1,1233,112,500.00,100.00,Denied,56900.0
2,2001,97110,1506.30,153.69,Denied,NaN
3,2002,97112,220.16,0.00,Denied,NaN
4,2003,97116,17.28,0.00,Denied,NaN
5,2003,97116,10.00,0.00,Denied,NaN
6,2004,97110,295.85,0.00,Denied,NaN


In [7]:
# 3. Group by ClaimNumber → join multiple focus codes
df_claims = (
    df_filtered
    .groupby('ClaimNumber')
    .agg({
        'FocusCode': lambda x: "|".join(sorted(set(x.astype(str)))),  # join codes with |
        'Pre_Reimb_Amount': 'sum',
        'Over_Payment': 'sum',
        'Decision_Type': 'first',   # pick first if consistent
        'Denial_Reason': 'first'
    })
    .reset_index()
)

In [7]:
print("Claims after merging focus codes:")
display(df_claims)

Claims after merging focus codes:


,ClaimNumber,FocusCode,Pre_Reimb_Amount,Over_Payment,Decision_Type,Denial_Reason
0,1233,111|112,1000.00,200.00,Denied,56900.0
1,2001,97110,1506.30,153.69,Denied,NaN
2,2002,97112,220.16,0.00,Denied,NaN
3,2003,97116,27.28,0.00,Denied,NaN
4,2004,97110,295.85,0.00,Denied,NaN


In [8]:
# 4. Calculate Total Error Dollar Amount (Case-level)
error_df = df_claims[~df_claims['Decision_Type'].isin(['Claim Accepted as Billed','Exclusion','Other'])]
total_error_dollars = error_df['Over_Payment'].sum()

print(f"Total Error Dollar Amount (Case-level): {total_error_dollars}")

Total Error Dollar Amount (Case-level): 353.69


In [9]:
# 5. Calculate Total Dollar Amount per Focus Code Combination
focus_totals = (
    df_claims
    .groupby('FocusCode')['Pre_Reimb_Amount']
    .sum()
    .reset_index()
    .rename(columns={'Pre_Reimb_Amount':'Total_Dollar_Amount'})
)

In [10]:
# 6. Apply same numerator to each focus code combination
focus_totals['Total_Error_Dollar'] = total_error_dollars
focus_totals['Error_Rate_%'] = (focus_totals['Total_Error_Dollar'] / focus_totals['Total_Dollar_Amount']) * 100

print("Error Rate Summary (by Focus Code Combination):")
display(focus_totals)

Error Rate Summary (by Focus Code Combination):


,FocusCode,Total_Dollar_Amount,Total_Error_Dollar,Error_Rate_%
0,111|112,1000.00,353.69,35.369000
1,97110,1802.15,353.69,19.626002
2,97112,220.16,353.69,160.651344
3,97116,27.28,353.69,1296.517595
